In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from causalinference import CausalModel

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
from linearmodels import IV2SLS




def Clean(data):
    
    new_df = pd.read_csv(data)       

    return new_df 
cwurData = "Input_Data/cwurData.csv" ##200 Nans in broad_impact column, 2200 rows/14 columns

clean_df = Clean(cwurData)

grouped_df = clean_df.groupby(['country','year']).mean().sort_values(by='country', ascending=False)
grouped_df





KeyboardInterrupt: 